In [ ]:
import numpy as np , matplotlib.pyplot as plt , seaborn as sns
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load The Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/sony-research/Data_Science_Challenge.csv')
df_copy = df.copy()
df.head()

* State: The state where a customer comes from
* Account length: Number of days a customer has been using services
* Area code: The area where a customer comes from
* Phone number: The phone number of a customer
* International Plan: The status of customer international plan
* Voicemail plan: The status of customer voicemail plan
* No. vmail msgs: Number of voicemail message sent by a customer
* Total day minutes: Total call minutes spent by a customer during day time
* Total day calls: Total number of calls made by a customer during day time
* Total day charge: Total amount charged to a customer during day time
* Total eve minutes: Total call minutes spent by a customer during evening time
* Total eve calls: Total number of calls made by a customer during evening time
* Total eve charge: Total amount charged to a customer during evening time
* Total night minutes: Total call minutes spent by a customer during night time
* Total night calls: Total number of calls made by a customer during night time
* Total night charge: Total amount charged to a customer during night time
* Total intl minutes: Total international call minutes spent by a customer
* Total intl calls: Total number of international calls made by a customer
* Total int charge: Total international call amount charged to a customer
* Customer service calls: Total number of customer service calls made by a customer
* churn: Whether a customer is churned or not

**Initial Inspection**
* Shape
* data types
* missing values
* duplicate values
* summary statistics
* Outlier detection
* Class balance

**Shape**

In [ ]:
df.shape

**Data Types**

In [ ]:
df.info()

**Missing Values**

In [ ]:
df.isna().sum()

**Duplicate Values**

In [ ]:
df.duplicated().sum()

**Summary Statistics**

In [ ]:
#Summary Statistics: Numerical variables
df.describe()

In [ ]:
#Summary Statistics: Categorical variables
df.describe(include = 'object')

# Outlier Detection

In [ ]:
def detect_outliers_iqr(df):
    outlier_count = {}
    for column in df.select_dtypes(include='number'):
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outlier_count[column] = outliers.shape[0]
    return outlier_count

outliers_iqr = detect_outliers_iqr(df)
outliers_df = pd.DataFrame(list(outliers_iqr.items()) , columns=['Column', 'Number of Outliers'])
outliers_df

# Class Balance

In [ ]:
class_balance = df['churn'].value_counts(normalize=True) * 100
print("Class Balance in dataFrame (as percentages):")
print(class_balance)

#plot the class balance
plt.figure(figsize=(6,4))
sns.countplot(x='churn' , data=df , palette='viridis')
plt.title('Class Distribution of Churn')
plt.xlabel('Churn')
plt.ylabel('Count')
plt.show()

Based on initial inspection, we can conclude that:
1. There are 3333 observations, each unique (no duplicates and all phone numbers unique), 20 features and 1 target variable(Churn).
2. None of the columns have any missing data.
3. The data types of all features are correct except "area code" which can be seen when we describe the numerical features. Mean "area code" doesn't make any sense and hence this data type must be changed in data pre-processing stage.
4. Most phone numbers belong to Wyoming (WY) state.
5. 3010 out of 3333 (90%) people don't have an international plan and 2411 out of 3333 (72%) don't have a voice mail plan.
6. Majority of columns have outliers. We need to handle them so that certain models don't get affected by outliers.
7. We have highly imbalanced dataset: 85.5% customers didn't churn while only 14.5% did.

Thus:
* This is a classification problem.
* Use model evaluation criteria other than accuracy (since randomly stating no customer will churn will give us 85.5% accuracy).Metrics like precision, recall, F1-score, ROC-AUC curve will become important.
* If identifying churned customers is important, make sure we perform oversampling/undersampling so that our model learns the minority class as well. Not doing this will lead to our model getting biased towards majority class.


# Data Processing
* Handling Missing Data: NOT Required
* Handling Duplicate Data: NOT Required
* Correct Data Types
* Drop features if required

# Correct Data Type


In [ ]:
#Correct Data type of area code
df['area code'] = df['area code'].astype('object')
df.describe(include='object')

Thus, we can see there are 3 area codes in the dataframe. However, 1655 out of 3333(~50%) belong to area code 415.

# Drop unwanted features
* Q: Will column phone number give us any information and should it be kept or dropped?

In [ ]:
df['phone number'].str.replace('-','').str.len().value_counts()

Thus, since every phone number is of same length, this column can't provide us much information and should be dropped.

In [ ]:
#Drop the 'phone number' column
df = df.drop('phone number',axis=1)
df.sample(n=5)

# Exploratory Data Analysis (EDA)
* Univariate analysis of numerical and categorical features.
* Multi-variate analysis to learn any correlations between features.
* Explore the relationships between features and the target variable (Churn).

In [ ]:
df.info()

# Distribution of numerical features

In [ ]:
numerical_features = df.select_dtypes(include=['float64','int64']).columns

fig , axes = plt.subplots(len(numerical_features)//3, 3, figsize=(18,20))
palette = sns.color_palette('Set2')
for i, col in enumerate(numerical_features):
    row , col_idx = divmod(i,3)
    sns.violinplot(x=col, data=df, ax=axes[row, col_idx],palette=palette)
    axes[row, col_idx].set_title(f'{col} Distribution')
    axes[row, col_idx].grid(True)
    
if len(numerical_features) % 3 != 0:
    for j in range(len(numerical_features) % 3,3):
        fig.delaxes(axes[-1,j])

plt.tight_layout()
plt.show()
    

We see that out of 15 numerical features, 3 features are not distributed normally, namely:
* number vmail messages
* total intl calls
* customer service calls

we can proceed in two ways:
1. Leave them as it is. Then only use tree-based models(XGBoost, Random Forest, Light GBM etc)since models like Logistic Regression, LDA, Naive Bayes, SVM etc assume normally distributed features.
2. Apply transformations so that we are free to use most of the models.

We will be applying transformations in the feature engineering step.


# Distribution of categorical features

In [ ]:
categorical_features = df.select_dtypes(include=['object']).columns
categorical_features = categorical_features.drop('state')

fig , axs = plt.subplots(1,3,figsize=(15,6))
axs = axs.flatten()

palette = sns.color_palette("Set2")

for i , feature in enumerate(categorical_features[:10]):
    plot = sns.countplot(x=df[feature],ax=axs[i],order=df[feature].value_counts().index, palette=palette)

    for p in plot.patches:
        plot.annotate(format(p.get_height(), '.0f'),(p.get_x() + p.get_width() / 2. , p.get_height()), ha ='center' , va = 'center' , xytext = (0 , -9), textcoords = 'offset points')

    axs[i].set_title(f'Distribution of {feature}')
    axs[i].set_xticklabels(axs[i].get_xticklabels(),rotation=45,horizontalalignment='right')

for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()


# Correlation Matrix & Pairplot for Numerical Features

In [ ]:
#correlation of numerical variables
pearson_correlation_matrix = df[numerical_features].corr(method = 'pearson')

plt.figure(figsize=(11,8))
sns.heatmap(pearson_correlation_matrix, annot=True, fmt=".2f",cmap='Dark2',linewidths=0.5)

plt.title('Pearson Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
palette = sns.color_palette("Set2")
plt.figure(figsize=(12,8))

def below_diag_pairplot(data,variables):
    data = data.copy()  
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    pd.set_option('future.no_silent_downcasting', True)

    
    g = sns.PairGrid(data[variables])
    g.map_lower(sns.scatterplot, s=15, color='skyblue')
    g.map_diag(sns.histplot, color='lightcoral', kde=True, palette = palette)

    for i , j in zip(*np.triu_indices_from(g.axes, 1)):
        g.axes[i,j].set_visible(False)
    plt.show()
below_diag_pairplot(df,numerical_features)

# Association Matrix of Categorical Features

In [ ]:
from scipy.stats import chi2_contingency

def cramers_v(x,y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r , k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r,k) - 1)))

cramers_v_matrix = pd.DataFrame(np.zeros((len(categorical_features), len(categorical_features))), index=categorical_features, columns=categorical_features)

for col1 in categorical_features:
    for col2 in categorical_features:
        if col1 != col2:
            cramers_v_matrix.loc[col1 , col2] = cramers_v(df[col1],df[col2])
        else:
            cramers_v_matrix.loc[col1 , col2] = np.nan

plt.figure(figsize = (6,5))
sns.heatmap(cramers_v_matrix, annot=True, cmap='Dark2', linewidths=0.5, fmt='.2f', cbar_kws={"shrink": .75})
plt.title("Cramer's V Association Matrix of Categorical Features")
plt.show()


We can see that we have few numerical features that are highly correlated:
* total day minutes & total day charge(100%)
* total eve minutes & total eve charge (100%)
* total night minutes & total night charge(100%)
* total intl minutes & total intl charge (100%)

we need to exclude some to avoid multi-collinearity. This will be done in *Feature selection* step.

# *Correlation of Features with Churn (Target Variable)*

In [ ]:
#Correlation of Numerical Features
plt.figure(figsize=(18,25))
for i , col in enumerate(numerical_features):
    plt.subplot(5,3,i+1)
    sns.boxplot(x='churn', y=col, data=df, palette=palette)
    plt.title(f'{col} vs Churn')
    plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
#categorical features and churn
churn_col = 'churn' #This should be the column indicating churn (True for churned, False for not churned)
plt.figure(figsize=(18,6))

for i , col in enumerate(categorical_features):
    plt.subplot(1, 3, i + 1)

    #calculate churn counts for each category
    churn_counts = df.groupby(col)[churn_col].value_counts(normalize=True).unstack()

    #calculate churn percentage (Percentage of True values)
    churn_counts['churn_percentage'] = churn_counts[True] * 100 #using True insted of 1
    
    #Resetting index for plotting
    churn_counts = churn_counts.reset_index()

    #Plotting
    ax = sns.barplot(x=col, y='churn_percentage',data=churn_counts, color='#FC8D62')

    plt.title(f'Churn Percentage by {col}')
    plt.xlabel(col)
    plt.ylabel('Churn Percentage')
    plt.xticks(rotation=45)

    #Adding percentages below the bars
    for p in ax.patches:
        percentage = f'{p.get_height():.2f}' #Format percentage to 2 decimal places
        ax.annotate(percentage, (p.get_x() + p.get_width() / 2., p.get_height()), ha='center' , va='bottom',fontsize=11, color='black', xytext=(0,-10), textcoords='offset points')

plt.tight_layout()
plt.show()

We can see that in numerical features, the following could be good predictors of churn:

* total day minutes
* total day charge (Churned customers have very high charges)
* total eve minutes
* total eve charge (Churned customers have moderately high charges)
* customer service calls (Churned customers have higher calls)

In categorical features, the following could be good predictors:

* Having international plan (4x more churn than NOT having)
* Having Voice mail plan (Half churn rate comapred to NOT having)